In [20]:
import pandas as pd
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
df_sample=pd.read_csv('sample_submission.csv')


In [21]:
df_train.head()

id  perc_premium_paid_by_cash_credit  age_in_days  Income  \
0  110936                             0.429        12058  355060   
1   41492                             0.010        21546  315150   
2   31300                             0.917        17531   84140   
3   19415                             0.049        15341  250510   
4   99379                             0.052        31400  198680   

   Count_3-6_months_late  Count_6-12_months_late  \
0                    0.0                     0.0   
1                    0.0                     0.0   
2                    2.0                     3.0   
3                    0.0                     0.0   
4                    0.0                     0.0   

   Count_more_than_12_months_late  application_underwriting_score  \
0                             0.0                           99.02   
1                             0.0                           99.89   
2                             1.0                           98.69   
3                             0.0                           99.57   
4                             0.0                           99.87   

   no_of_premiums_paid sourcing_channel residence_area_type  premium  renewal  
0                   13                C               Urban     3300        1  
1                   21                A               Urban    18000        1  
2                    7                C               Rural     3300        0  
3                    9                A               Urban     9600        1  
4                   12                B               Urban     9600        1

In [22]:
df_test.head()

id  perc_premium_paid_by_cash_credit  age_in_days  Income  \
0    649                             0.001        27384   51150   
1  81136                             0.124        23735  285140   
2  70762                             1.000        17170  186030   
3  53935                             0.198        16068  123540   
4  15476                             0.041        10591  200020   

   Count_3-6_months_late  Count_6-12_months_late  \
0                    0.0                     0.0   
1                    0.0                     0.0   
2                    0.0                     0.0   
3                    0.0                     0.0   
4                    1.0                     0.0   

   Count_more_than_12_months_late  application_underwriting_score  \
0                             0.0                           99.89   
1                             0.0                           98.93   
2                             0.0                             NaN   
3                             0.0                           99.00   
4                             0.0                           99.17   

   no_of_premiums_paid sourcing_channel residence_area_type  premium  
0                    7                A               Rural     3300  
1                   19                A               Urban    11700  
2                    2                B               Urban    11700  
3                   11                B               Rural     5400  
4                   14                A               Rural     9600

In [23]:
df_sample.head()

id  renewal  incentives
0    649      0.5        1650
1  81136      0.5        1650
2  70762      0.5        1650
3  53935      0.5        1650
4  15476      0.5        1650

In [24]:
df_train.isnull().sum()

id                                     0
perc_premium_paid_by_cash_credit       0
age_in_days                            0
Income                                 0
Count_3-6_months_late                 97
Count_6-12_months_late                97
Count_more_than_12_months_late        97
application_underwriting_score      2974
no_of_premiums_paid                    0
sourcing_channel                       0
residence_area_type                    0
premium                                0
renewal                                0
dtype: int64

In [25]:
df_train['Count_3-6_months_late'].fillna(0,inplace=True)
df_test['Count_3-6_months_late'].fillna(0,inplace=True)

In [26]:
df_train['Count_6-12_months_late'].fillna(0,inplace=True)
df_test['Count_6-12_months_late'].fillna(0,inplace=True)

In [27]:
df_train['Count_more_than_12_months_late'].fillna(0,inplace=True)
df_test['Count_more_than_12_months_late'].fillna(0,inplace=True)

In [28]:
df_train['application_underwriting_score'].fillna(df_train['application_underwriting_score'].mean(),inplace=True)
df_test['application_underwriting_score'].fillna(df_test['application_underwriting_score'].mean(),inplace=True)

In [29]:
renewal=df_train['renewal']
df_train=df_train.drop(['renewal'],axis=1)
df_total=pd.concat([df_train,df_test])



In [30]:
df_total=pd.get_dummies(df_total,columns=['sourcing_channel'],drop_first=True)
df_total=pd.get_dummies(df_total,columns=['residence_area_type'],drop_first=True)

In [31]:
shape=df_train.shape[0]

In [32]:
df_total.head()

id  perc_premium_paid_by_cash_credit  age_in_days  Income  \
0  110936                             0.429        12058  355060   
1   41492                             0.010        21546  315150   
2   31300                             0.917        17531   84140   
3   19415                             0.049        15341  250510   
4   99379                             0.052        31400  198680   

   Count_3-6_months_late  Count_6-12_months_late  \
0                    0.0                     0.0   
1                    0.0                     0.0   
2                    2.0                     3.0   
3                    0.0                     0.0   
4                    0.0                     0.0   

   Count_more_than_12_months_late  application_underwriting_score  \
0                             0.0                           99.02   
1                             0.0                           99.89   
2                             1.0                           98.69   
3                             0.0                           99.57   
4                             0.0                           99.87   

   no_of_premiums_paid  premium  sourcing_channel_B  sourcing_channel_C  \
0                   13     3300                   0                   1   
1                   21    18000                   0                   0   
2                    7     3300                   0                   1   
3                    9     9600                   0                   0   
4                   12     9600                   1                   0   

   sourcing_channel_D  sourcing_channel_E  residence_area_type_Urban  
0                   0                   0                          1  
1                   0                   0                          1  
2                   0                   0                          0  
3                   0                   0                          1  
4                   0                   0                          1

In [33]:
df_total=df_total.drop(['perc_premium_paid_by_cash_credit','age_in_days','application_underwriting_score'])

In [34]:
df_total_train=df_total.iloc[:shape]
df_total_test=df_total.iloc[shape:]
X=df_total_train.values
y=renewal.values
test=df_total_test.values

In [35]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state = 42)
from pprint import pprint
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [36]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [37]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X,y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True 
[CV] n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True 
[CV] n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True 
[CV] n_estimators=2000, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True 
[CV]  n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True, total= 1.3min
[CV] n_estimators=2000, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True 
[CV]  n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True, total= 1.4min
[CV] n_estimators=2000, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True 


Process ForkPoolWorker-6:
KeyboardInterrupt
Traceback (most recent call last):
  File "/home/himanshun3/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/himanshun3/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/himanshun3/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/himanshun3/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/pool.py", line 360, in get
    racquire()
KeyboardInterrupt
Process ForkPoolWorker-7:
Traceback (most recent call last):
  File "/home/himanshun3/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-5:
  File "/home/himanshun3/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/himanshun3/anaconda3/lib/python3.6/multiproces

KeyboardInterrupt: 

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_params_'

In [93]:
from sklearn.ensemble import RandomForestClassifier
y_pred_new_2=[]
rf = RandomForestClassifier(n_estimators = 2000,max_depth=10,min_samples_split=10,min_samples_leaf=4,max_features='sqrt',bootstrap=True)
rf.fit(X,y)
y_pred=rf.predict_proba(test)


In [97]:
for i in range(len(y_pred)):
    y_pred_new_2.append(y_pred[i][1])
    
df_test['renewal']=y_pred_new_2

ValueError: Length of values does not match length of index

In [102]:
df_test['incentives']=0.02750*df_test['premium']
df_test[['id','renewal','incentives']].to_csv('submission_15.csv',index=False)

In [103]:
df_result=pd.read_csv('submission_15.csv')
df_result

id   renewal  incentives
0         649  0.990625       90.75
1       81136  0.982812      321.75
2       70762  0.827389      321.75
3       53935  0.977199      148.50
4       15476  0.952160      264.00
5       64797  0.982653      321.75
6       67412  0.849359       90.75
7       44241  0.844157      148.50
8        5069  0.988623      379.50
9       16615  0.990373      783.75
10      84281  0.983353      552.75
11      82623  0.968233       33.00
12      86475  0.644240      321.75
13      68263  0.944169       33.00
14     113135  0.944236       33.00
15      45532  0.971009      495.00
16     102161  0.990738      206.25
17      84789  0.696764      841.50
18      62398  0.974378      321.75
19      70148  0.974964       90.75
20     102487  0.918495      379.50
21     100513  0.908508      321.75
22      92702  0.913223      156.75
23      25051  0.662558      264.00
24      92920  0.990271      379.50
25      25520  0.965971      321.75
26      81869  0.968061     1072.50
27      75402  0.978464      206.25
28      34053  0.679174       33.00
29      21191  0.984122      264.00
...       ...       ...         ...
34194   79950  0.985368      206.25
34195   30272  0.990079       33.00
34196   85914  0.992192      379.50
34197   10922  0.984566      379.50
34198   93046  0.984372      148.50
34199   52826  0.944707      495.00
34200   35994  0.983108      841.50
34201    1512  0.985966      148.50
34202   52077  0.972770      206.25
34203   95749  0.976188      206.25
34204   49946  0.981704       90.75
34205   66811  0.981142      264.00
34206   53438  0.987203       33.00
34207   24435  0.983191      552.75
34208   17848  0.976711      379.50
34209   53626  0.981236      148.50
34210   87268  0.989446      379.50
34211   58450  0.978989       33.00
34212   48188  0.986535      668.25
34213   44239  0.976668      321.75
34214   24582  0.934576      206.25
34215   79939  0.965646       90.75
34216   63759  0.971328       33.00
34217   63725  0.719995       90.75
34218   37116  0.963726      610.50
34219   81988  0.967352      148.50
34220   30326  0.981774       33.00
34221   65749  0.960387       33.00
34222   65686  0.735292      379.50
34223   93826  0.961813      148.50

[34224 rows x 3 columns]